In [1]:
from sys import path as wd
from os import path, getcwd
from time import sleep
import pandas as pd

workdir = path.dirname(getcwd())
wd.append(workdir)

from dags.operadores import (GoogleSheetManager, GoogleDriveManager)
from dags.estruturas import (InformationManager)
from dags.configure_directory import (DirectoryManager)
from dags.sidrapi import SidraManager
import json

C:\Users\usuario\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


c:\Users\usuario\OneDrive\Documentos\R\Banco de Dados\Sidra-PI V2\Sidra-PI V2\data


In [2]:
data_json = path.join(workdir, "data", "__data__info.json")
local_dir = path.join(workdir, "data", "processed - Copia")
final_dir = path.join(workdir, "data", "formatted")

with open(data_json, 'r') as file:
    info_data = json.load(file)

#
credentials_dir = path.join(workdir, "data", "credentials.json")
knowledge_sheet_url = r'https://docs.google.com/spreadsheets/d/11-CI4nLXTte0OcDVMl-My4PtOdif7ziWB0GKg6hkJMw/edit?usp=sharing'

In [4]:
# Classes
sheet_manager = GoogleSheetManager(credentials_dir)
drive_manager = GoogleDriveManager(credentials_dir, data_json)
information_manager = InformationManager(knowledge_sheet_url)
directory_manager = DirectoryManager(local_dir, final_dir)
sidra_manager = SidraManager()

In [5]:
df = directory_manager._list_files()
df['Tabela'] = df['filename'].str.extract('(\d+)')
df['Caminho'] = df['filename'].apply(lambda x: f'{local_dir}//{x}')
df = df[df['Tabela'].notnull()]
df

,filename,Tabela,Caminho
3,Tabela 1086.xlsx,1086,c:\Users\usuario\OneDrive\Documentos\R\Banco d...
4,Tabela 1088.xlsx,1088,c:\Users\usuario\OneDrive\Documentos\R\Banco d...
5,Tabela 1089.xlsx,1089,c:\Users\usuario\OneDrive\Documentos\R\Banco d...
6,Tabela 109.xlsx,109,c:\Users\usuario\OneDrive\Documentos\R\Banco d...
7,Tabela 1090.xlsx,1090,c:\Users\usuario\OneDrive\Documentos\R\Banco d...
...,...,...,...
2049,Tabela 992.xlsx,992,c:\Users\usuario\OneDrive\Documentos\R\Banco d...
2050,Tabela 993.xlsx,993,c:\Users\usuario\OneDrive\Documentos\R\Banco d...
2051,Tabela 994.xlsx,994,c:\Users\usuario\OneDrive\Documentos\R\Banco d...
2052,Tabela 995.xlsx,995,c:\Users\usuario\OneDrive\Documentos\R\Banco d...


In [7]:
template = path.join(local_dir, "..", "template.xlsx")

for idx, row in df.iterrows():
    try:
        # Tenta obter os metadados e processar a tabela SIDRA
        json_data = sidra_manager.sidra_get_metadata(row['Tabela'])
        _, df_sidra = sidra_manager.sidra_process_table(json_data)

        print(f'tabela {row["Tabela"]} ajustada')
        directory_manager.process_template(df_sidra, template, row['Caminho'])

    except Exception as e:
        print(f"Erro ao processar a tabela {row['Tabela']}: {e}")

tabela 1086 ajustada
tabela 1088 ajustada
tabela 1089 ajustada
tabela 109 ajustada
tabela 1090 ajustada
tabela 1092 ajustada
tabela 1093 ajustada
tabela 1094 ajustada
tabela 1209 ajustada
tabela 1263 ajustada
tabela 1264 ajustada
tabela 1265 ajustada
tabela 1266 ajustada
tabela 1594 ajustada
tabela 1595 ajustada
tabela 1599 ajustada
tabela 1608 ajustada
tabela 1609 ajustada
tabela 1616 ajustada
tabela 1618 ajustada
tabela 1622 ajustada
tabela 1627 ajustada
tabela 1629 ajustada
tabela 1647 ajustada
tabela 1650 ajustada
tabela 1651 ajustada
tabela 1652 ajustada
tabela 1655 ajustada
tabela 1656 ajustada
tabela 1664 ajustada
tabela 1685 ajustada
tabela 1732 ajustada
tabela 1733 ajustada
tabela 1734 ajustada
tabela 1783 ajustada
tabela 1799 ajustada
tabela 1802 ajustada
tabela 1803 ajustada
tabela 1805 ajustada
tabela 1807 ajustada
tabela 1809 ajustada
tabela 1810 ajustada
tabela 1811 ajustada
tabela 1818 ajustada
tabela 188 ajustada
tabela 209 ajustada
tabela 21 ajustada
tabela 2286 ajusta

In [8]:
directory_manager

In [9]:
df_information = sheet_manager.get_data_info_from_url(knowledge_sheet_url)

In [10]:
dm = DirectoryManager(local_dir, final_dir)
dm.execute_organize_files(df_information)

In [11]:
t = drive_manager.get_file_info_by_url(r'https://drive.google.com/drive/u/0/folders/1KSA0jimLRA1ub990ud5ve8oPd5aN_QPH')
t

{'id': '1KSA0jimLRA1ub990ud5ve8oPd5aN_QPH',
 'name': 'Banco de Dados',
 'mimeType': 'application/vnd.google-apps.folder'}

In [12]:
content = drive_manager.list_all_files('1KSA0jimLRA1ub990ud5ve8oPd5aN_QPH')

In [13]:
content

[{'id': '1C5j9P_pY3nzfUpF7PmDGmvRfWXnHHdbZ',
  'name': 'Tabela 9464.xlsx',
  'type': 'file'},
 {'id': '1RHah1d7bBZ3cdDV8Flea8qsYyE1_XSqV',
  'name': 'Tabela 8157.xlsx',
  'type': 'file'},
 {'id': '1AaBlwjYBEvyOhfbiXX4lcFQbyZG8BhEY',
  'name': 'Tabela 7190.xlsx',
  'type': 'file'},
 {'id': '1yIsIr8maTewrR2TcO3JXf6g0Tf-ITrk1',
  'name': 'Tabela 7191.xlsx',
  'type': 'file'},
 {'id': '1kPzhoEXlB0yi3eeno5rNbx0PbgZ-RiTe',
  'name': 'Tabela 7192.xlsx',
  'type': 'file'},
 {'id': '13dJNHCXegQ18C5y0DgCbxc1EEqtJKC4G',
  'name': 'Tabela 6976.xlsx',
  'type': 'file'},
 {'id': '1hLTC6eeLMtQ87LsI2rO4mLw-msIQ9Yoi',
  'name': 'Tabela 6970.xlsx',
  'type': 'file'},
 {'id': '1zGuP2Uf14_SPNZJIccKYkh3KXiA5Z8w4',
  'name': 'Tabela 6822.xlsx',
  'type': 'file'},
 {'id': '1mPDq9LJVErwghdWWInP-RcEoH_jeM7pO',
  'name': 'Tabela 6824.xlsx',
  'type': 'file'},
 {'id': '17_8nH3R_SDhbLhhyCxclvvFc4qnwlRFq',
  'name': 'Tabela 6823.xlsx',
  'type': 'file'},
 {'id': '1JisVOQNTbQvTsK30sqXien909YWW2tSX',
  'name': 'Tabe

In [17]:
df = pd.json_normalize(content)
df['url'] = df['id'].apply(lambda x: f'https://docs.google.com/spreadsheets/d/{x}/view')
df['tabela'] = df['name'].str.extract('(\d+)')
df.to_excel(path.join(r'C:\Users\usuario\OneDrive\Documentos\R\Banco de Dados\Sidra-PI V2\Sidra-PI V2\data', "tabelas_link.xlsx"))

In [16]:
df

,id,name,type,url,tabela
0,1C5j9P_pY3nzfUpF7PmDGmvRfWXnHHdbZ,Tabela 9464.xlsx,file,https://drive.google.com/file/d/1C5j9P_pY3nzfU...,9464
1,1RHah1d7bBZ3cdDV8Flea8qsYyE1_XSqV,Tabela 8157.xlsx,file,https://drive.google.com/file/d/1RHah1d7bBZ3cd...,8157
2,1AaBlwjYBEvyOhfbiXX4lcFQbyZG8BhEY,Tabela 7190.xlsx,file,https://drive.google.com/file/d/1AaBlwjYBEvyOh...,7190
3,1yIsIr8maTewrR2TcO3JXf6g0Tf-ITrk1,Tabela 7191.xlsx,file,https://drive.google.com/file/d/1yIsIr8maTewrR...,7191
4,1kPzhoEXlB0yi3eeno5rNbx0PbgZ-RiTe,Tabela 7192.xlsx,file,https://drive.google.com/file/d/1kPzhoEXlB0yi3...,7192
...,...,...,...,...,...
1728,1ALKHntx_7N37rOzQxHsOGKhZbih-VcKJ,Tabela 7108.xlsx,file,https://drive.google.com/file/d/1ALKHntx_7N37r...,7108
1729,1vGERNjV0QifRLNLDUU8Qqyv-xLgbBWDr,Tabela 7130.xlsx,file,https://drive.google.com/file/d/1vGERNjV0QifRL...,7130
1730,15UQsKd7gVO4l5kciD_JG3NVuPyxbgIUX,Tabela 7107.xlsx,file,https://drive.google.com/file/d/15UQsKd7gVO4l5...,7107
1731,1dqjsNCM8Zng0mB2DamtjXtdPZPu0bMbD,Tabela 6412.xlsx,file,https://drive.google.com/file/d/1dqjsNCM8Zng0m...,6412
